<a href="https://colab.research.google.com/github/NachoDave/Automated-Vehicle-Anomalous-Driving-Behaviour/blob/main/CAMs_Autoencoder_project_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoder project

## What do we need to do?


1.   Get and format the data
2.   Split the data into training, test and validation sets - initially will just add the data one measure at a time, but may need to combine them somehow
3. Define the model
4. Crea



### Import Libraries and connect to drive

In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

from google.colab import drive

drive.mount('/content/drive')
dataset_dir = '/content/drive/MyDrive/datasets/car_hacking'


import sys
sys.path.append(dataset_dir)

from utils import *




Mounted at /content/drive


### Load the datasets

In [2]:
## Function to load the normal da


# Load normal data
normal_df = load_canbus_text_file(f"{dataset_dir}/normal_run_data.txt")
print(normal_df.head())
#normal_df.to_csv(f"{dataset_dir}/normal_run_data.csv", index=False)

# DoS data


# Fuzzy Data

# RPM Data
rpm_df = pd.read_csv(f"{dataset_dir}/RPM_dataset.csv", header=None)
print(rpm_df.head())

# Gear dataset
gear_df = pd.read_csv(f"{dataset_dir}/gear_dataset.csv", header=None, on_bad_lines='skip')
print(gear_df.head())



           Timestamp    ID  DLC Byte1 Byte2 Byte3 Byte4 Byte5 Byte6 Byte7  \
0  1479121434.850202  0350    8    05    28    84    66    6d    00    00   
1  1479121434.850423  02c0    8    14    00    00    00    00    00    00   
2  1479121434.850977  0430    8    00    00    00    00    00    00    00   
3  1479121434.851215  04b1    8    00    00    00    00    00    00    00   
4  1479121434.851463  01f1    8    00    00    00    00    00    00    00   

  Byte8  
0    a2  
1    00  
2    00  
3    00  
4    00  
             0     1   2   3   4   5   6   7   8   9   10 11
0  1.478191e+09  0316   8  05  22  68  09  22  20  00  75  R
1  1.478191e+09  018f   8  fe  3b  00  00  00  3c  00  00  R
2  1.478191e+09  0260   8  19  22  22  30  ff  8f  6e  3f  R
3  1.478191e+09  02a0   8  60  00  83  1d  96  02  bd  00  R
4  1.478191e+09  0329   8  dc  b8  7e  14  11  20  00  14  R


/tmp/ipython-input-190102036.py:19: DtypeWarning: Columns (3,4,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  gear_df = pd.read_csv(f"{dataset_dir}/gear_dataset.csv", header=None, on_bad_lines='skip')


             0     1   2   3   4   5   6   7   8   9   10 11
0  1.478193e+09  0140   8  00  00  00  00  10  29  2a  24  R
1  1.478193e+09  02c0   8  15  00  00  00  00  00  00  00  R
2  1.478193e+09  0350   8  05  20  44  68  77  00  00  7e  R
3  1.478193e+09  0370   8  00  20  00  00  00  00  00  00  R
4  1.478193e+09  043f   8  10  40  60  ff  78  c4  08  00  R


In [3]:
normal_df['ID'].value_counts()

,count
ID,
02c0,50689
018f,50687
0260,50687
02a0,50687
0316,50687
0329,50686
0545,50686
0350,50661
0131,50647


In [4]:
gear_df.sample(10000)[1].value_counts()#.nunique()#, rpm_df[1].nunique(), normal_df['ID'].nunique()

,count
1,
043f,2080
0153,500
018f,485
0440,463
0002,461
0140,455
0316,453
02c0,448
0545,446


In [5]:
# Load rpm dataset
rpm_df = pd.read_csv(f"{dataset_dir}/RPM_dataset.csv", header=None)
rpm_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.478191e+09,0316,8,05,22,68,09,22,20,00,75,R
1,1.478191e+09,018f,8,fe,3b,00,00,00,3c,00,00,R
2,1.478191e+09,0260,8,19,22,22,30,ff,8f,6e,3f,R
3,1.478191e+09,02a0,8,60,00,83,1d,96,02,bd,00,R
4,1.478191e+09,0329,8,dc,b8,7e,14,11,20,00,14,R


In [6]:
rpm_df[1].unique().shape

(26,)

In [7]:
# Load Fuzzy attack dataset
DoS_df = pd.read_csv(f"{dataset_dir}/DoS_dataset.csv", header=None)
DoS_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.478198e+09,0316,8,05,21,68,09,21,21,00,6f,R
1,1.478198e+09,018f,8,fe,5b,00,00,00,3c,00,00,R
2,1.478198e+09,0260,8,19,21,22,30,08,8e,6d,3a,R
3,1.478198e+09,02a0,8,64,00,9a,1d,97,02,bd,00,R
4,1.478198e+09,0329,8,40,bb,7f,14,11,20,00,14,R


In [8]:
fuzz_df = pd.read_csv(f"{dataset_dir}/Fuzzy_dataset.csv", header=None, on_bad_lines='skip')
fuzz_df.head()

/tmp/ipython-input-2567828019.py:1: DtypeWarning: Columns (6,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  fuzz_df = pd.read_csv(f"{dataset_dir}/Fuzzy_dataset.csv", header=None, on_bad_lines='skip')


,0,1,2,3,4,5,6,7,8,9,10,11
0,1.478196e+09,0545,8,d8,00,00,8a,00,00,00,00,R
1,1.478196e+09,02b0,5,ff,7f,00,05,49,R,NaN,NaN,NaN
2,1.478196e+09,0002,8,00,00,00,00,00,01,07,15,R
3,1.478196e+09,0153,8,00,21,10,ff,00,ff,00,00,R
4,1.478196e+09,0130,8,19,80,00,ff,fe,7f,07,60,R


In [23]:
fuzz_df[1].unique(), DoS_df[1].unique(), rpm_df[1].unique(),normal_df['ID'].unique()

(array(['0545', '02b0', '0002', ..., '0524', '00d6', '0038'], dtype=object),
 array(['0316', '018f', '0260', '02a0', '0329', '0545', '0002', '0153',
        '02c0', '0130', '0131', '0140', '0350', '043f', '0370', '0440',
        '04f0', '0430', '04b1', '01f1', '05f0', '00a0', '00a1', '0690',
        '05a0', '05a2', '0000'], dtype=object),
 array(['0316', '018f', '0260', '02a0', '0329', '0545', '02c0', '0002',
        '0153', '0130', '0131', '0140', '043f', '0370', '0440', '0350',
        '04f0', '0430', '04b1', '01f1', '0690', '05f0', '00a0', '00a1',
        '05a0', '05a2'], dtype=object),
 array(['0350', '02c0', '0430', '04b1', '01f1', '0153', '0002', '018f',
        '0130', '0131', '0140', '0260', '02a0', '0316', '0329', '0545',
        '02b0', '043f', '0370', '0440', '04f0', '05f0', '05a0', '05a2',
        '0690', '00a0', '00a1'], dtype=object))

In [10]:
fuzz_df[0:5]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.478196e+09,0545,8,d8,00,00,8a,00,00,00,00,R
1,1.478196e+09,02b0,5,ff,7f,00,05,49,R,NaN,NaN,NaN
2,1.478196e+09,0002,8,00,00,00,00,00,01,07,15,R
3,1.478196e+09,0153,8,00,21,10,ff,00,ff,00,00,R
4,1.478196e+09,0130,8,19,80,00,ff,fe,7f,07,60,R


In [ ]:
## Get the bus numbers


In [20]:
## Value counts for the CAN Bus IDs
#normal_df['ID'].value_counts() #> 50000

high_freq_entries = normal_df['ID'].value_counts()
normal_can_bus_IDs = high_freq_entries[high_freq_entries > 50000].index.tolist()

normal_can_bus_IDs


['02c0',
 '018f',
 '0260',
 '02a0',
 '0316',
 '0329',
 '0545',
 '0350',
 '0131',
 '0130',
 '0002',
 '0153',
 '0140',
 '0370',
 '043f',
 '0440',
 '02b0']

In [82]:
labelled_cans = normal_can_bus_IDs

ddf = normal_df[0:100]
ddf = fuzz_df[0:10]


def format_CAN_bus_data(df:pd.DataFrame, labelled_cans) -> pd.DataFrame:
  # Rows with only two bytes have nans
  df = df.iloc[:,0:11] # remove the last column
  df.loc[~(df.iloc[:,1].isin(labelled_cans)), df.columns[1]] = 9999 # any bus ids not in the main list

  one_hot_ids = labelled_cans + ['9999']
  one_hot_ids

  df.replace(to_replace = np.nan, value = '00', inplace=True) # replace the NaNs
  df.replace(to_replace = 'R', value = '00', inplace=True) # replace the R Values in the 5 column for the 2 byte channels

  # Get the bus Ids and convert to numbers then to one hot encoding
  df[df.columns[1]] = pd.Categorical(df[df.columns[1]], categories=one_hot_ids)
  bus_id_one_hot = pd.get_dummies(df[df.columns[1]], columns=one_hot_ids).astype(int)

  # Get the payload and convert to int
  payload = df.iloc[:,3:11].applymap(int,base=16)/255

  return pd.concat([bus_id_one_hot, payload], axis=1)

format_CAN_bus_data(ddf, labelled_cans)

/tmp/ipython-input-4171389312.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payload = df.iloc[:,3:11].applymap(int,base=16)/255


,02c0,018f,0260,02a0,0316,0329,0545,0350,0131,0130,...,02b0,9999,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0.847059,0.000000,0.000000,0.541176,0.000000,0.000000,0.000000,0.000000
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1.000000,0.498039,0.000000,0.019608,0.286275,0.000000,0.000000,0.000000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.003922,0.027451,0.082353
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.129412,0.062745,1.000000,0.000000,1.000000,0.000000,0.000000
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0.098039,0.501961,0.000000,1.000000,0.996078,0.498039,0.027451,0.376471
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0.090196,0.501961,0.000000,0.000000,0.396078,0.498039,0.027451,0.623529
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.000000,0.000000,0.000000,0.007843,0.125490,0.152941,0.658824
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0.019608,0.125490,0.078431,0.407843,0.470588,0.000000,0.000000,0.129412
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0.082353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0.125490,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Format the data

In [ ]:
def format_CAN_bus_data(df:pd.DataFrame) -> pd.DataFrame:
  # Rows with only two bytes have nans
  df.replace(to_replace = np.nan, value = '00', inplace=True) # replace the NaNs
  df[5].replace(to_replace = 'R', value = '00', inplace=True) # replace the R Values in the 5 column for the 2 byte channels

  # Get the bus Ids and convert to numbers then to one hot encoding
  bus_id = df[1].apply(int,base=16)
  bus_id_one_hot = pd.get_dummies(bus_id).astype(int)

  # Get the payload and convert to int
  payload = df.iloc[:,3:11].applymap(int,base=16)/255

  # Concat the bus_id and payload to give the final data frame
  return pd.concat([bus_id_one_hot, payload], axis=1)

rpm_df = format_CAN_bus_data(rpm_df)
rpm_df.head()

/tmp/ipython-input-462025556.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[5].replace(to_replace = 'R', value = '00', inplace=True) # replace the R Values in the 5 column for the 2 byte channels
/tmp/ipython-input-462025556.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payload = df.iloc[:,3:11].applymap(int,base=16)/255


,2,160,161,304,305,320,339,399,497,608,...,1520,1680,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.019608,0.133333,0.407843,0.035294,0.133333,0.125490,0.000000,0.458824
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0.996078,0.231373,0.000000,0.000000,0.000000,0.235294,0.000000,0.000000
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0.098039,0.133333,0.133333,0.188235,1.000000,0.560784,0.431373,0.247059
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.376471,0.000000,0.513725,0.113725,0.588235,0.007843,0.741176,0.000000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0.862745,0.721569,0.494118,0.078431,0.066667,0.125490,0.000000,0.078431


In [ ]:
fuzz_format_df = format_CAN_bus_data(fuzz_df)
fuzz_format_df.head()

/tmp/ipython-input-462025556.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[5].replace(to_replace = 'R', value = '00', inplace=True) # replace the R Values in the 5 column for the 2 byte channels


In [ ]:
normal_format_df = format_CAN_bus_data(normal_df)
normal_format_df.head()

KeyError: 5

### Get data ready for loading

In [ ]:
X = rpm_df.values.astype("float32")   # your dataframe of 34 features
X_train, X_temp = train_test_split(X, test_size=0.3, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)

In [ ]:
X_train.shape

(3235191, 34)

### Create the autoencoder

In [ ]:
input_dim = X_train.shape[1]   # should be 34
encoding_dim = 16              # latent size (tune this)

input_layer = layers.Input(shape=(input_dim,)) # this is the
encoder = layers.Dense(64, activation="relu")(input_layer)
encoder = layers.Dense(32, activation="relu")(encoder)
latent = layers.Dense(encoding_dim, activation="relu")(encoder)

decoder = layers.Dense(32, activation="relu")(latent)
decoder = layers.Dense(64, activation="relu")(decoder)
output_layer = layers.Dense(input_dim, activation="sigmoid")(decoder)

autoencoder = models.Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 34)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 34)             │         2,210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,714 (37.95 KB)

 Trainable params: 9,714 (37.95 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train[0:100]

array([[  1.,   0.,   0., ...,   8.,   5.,  57.],
       [  0.,   1.,   0., ...,  31.,   2.,   0.],
       [  0.,   0.,   0., ...,  36.,   0., 255.],
       ...,
       [  0.,   0.,   0., ...,  36.,   0., 255.],
       [  0.,   0.,   0., ...,  36.,   0., 255.],
       [  0.,   0.,   0., ...,   2., 189.,   0.]], dtype=float32)

### Train the model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
x = 200000
history = autoencoder.fit(
    X_train[0:x], X_train[0:x],
    epochs=20,
    batch_size=128,
    shuffle=True,
    validation_data=(X_val[0:x], X_val[0:x]),
    #callbacks=[callback]
)

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0392 - val_loss: 0.0070
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0065 - val_loss: 0.0062
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0062 - val_loss: 0.0062
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0054 - val_loss: 0.0049
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0048 - val_loss: 0.0049
Epoch 9/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0048 - val_loss: 0.0049
Epoch 10/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0048 - val_loss: 0.0047
Epoch 11/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.0047 - val_loss: 0.0047
Epoch 12/20
1563/1563 ━━━

In [ ]:
X_val[0] - autoencoder.predict(X_val[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[ 3.75032425e-04, -6.31413005e-20, -4.76683172e-11,
        -1.19724684e-27, -6.23911205e-07, -4.13691259e-09,
        -3.49473231e-34, -4.95110442e-20, -1.33580137e-12,
        -1.14108538e-13, -6.05412553e-10, -1.80433723e-04,
        -9.48745138e-14, -3.88414171e-17, -1.96964329e-08,
        -1.57185452e-04, -1.34581400e-11, -6.54869876e-27,
        -3.89161712e-13, -1.37685705e-11, -1.38074280e-20,
        -1.37555051e-10, -3.72776893e-10, -6.27334434e-11,
        -4.27120284e-10, -2.96111855e-12, -3.69213708e-08,
        -3.99012864e-03, -2.41975727e-06, -2.33875835e-06,
        -8.11789709e-04,  2.33866740e-02,  4.70507294e-02,
         9.98006761e-03]], dtype=float32)